In [1]:
import pandas as pd

In [2]:
retail = pd.read_csv('C:/Users/bkunt/jupyter/karpov/5_data.csv.zip', compression='zip', encoding='ISO-8859-1')

In [3]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
retail.shape

(541909, 8)

In [5]:
#проверим, сколько дубликатов в данных
retail.duplicated().sum()

5268

In [6]:
#удалим дубликаты
retail.drop_duplicates(inplace=True)
retail.shape

(536641, 8)

In [7]:
#узнаем, сколько транзакций отменили пользователи (InvoiceNo начинается с буквы 'C')
retail.InvoiceNo.str.startswith('C').sum()

9251

In [8]:
#определим заказы, в которых количество товара не равно 0
retail.query('Quantity > 0').shape

(526054, 8)

In [9]:
#найдем пользователей из Германии с числом заказов выше 80%-го процентиля 
germany_top = retail.query('Country == "Germany"') \
                    .groupby('CustomerID', as_index=False) \
                    .agg({'InvoiceNo': 'nunique'}) \
                    .rename(columns={'InvoiceNo': 'Orders'}) \
                    .query('Orders > Orders.quantile(q=0.8)')['CustomerID']

In [14]:
#получим остальную информацию по этим пользователям
top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


In [18]:
#самый популярный товар среди пользователей в Германии
top_retail_germany.query('StockCode != "POST"') \
                  .groupby('StockCode') \
                  .agg({'Quantity': 'count'}) \
                  .sort_values('Quantity', ascending=False).head()

,Quantity
StockCode,
22326,62
22423,55
22328,45
22467,37
22077,34


In [28]:
#топ-5 товаров по сумме заказа
retail['Revenue'] = retail.Quantity * retail.UnitPrice
retail.groupby('InvoiceNo', as_index=False) \
      .agg({'Revenue': 'sum'}) \
      .rename(columns={'Revenue': 'TotalRevenue'}) \
      .sort_values('TotalRevenue', ascending=False)['InvoiceNo'].head().to_list()

['581483', '541431', '574941', '576365', '556444']